In [1]:
import matplotlib
import globus_sdk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

## MDF and DLHub imports
from mdf_forge.forge import Forge
import mdf_toolbox
from dlhub_client.client import DLHub

In [2]:
mdf = Forge()
dl = DLHub()
creds = mdf_toolbox.login(app_name="DLHub", services=["petrel", "data_mdf", "transfer"])

---
A classification model for x-ray diffraction images

Here we use a deep learning model developed by [Kevin Yager and colleagues](https://doi.org/10.1109/NYSDS.2017.8085045) at Brookhaven National Laboratory that uses residual deep learning networks to assign useful tags to small angle scattering (SAXS) images. Such a model might be used from a light source beamline, for example, to validate data as they are collected.

The servable is set up to retrieve a specified image from Petrel, a multi-petabyte data store at Argonne used to hold data from Advanced Photon Source experiments, and then pass the image through the Yager model.

![resnet](img/resnet_1.png)

## Get a list of data from Petrel

Use the Globus Transfer token to list the contents of a directory on Petrel. 

In [3]:
target_endpoint = "e38ee745-6d04-11e5-ba46-22000b92c6ec"
target_path = "Yager/model/raw/01470b9d_varied_sm/"

ls_result = creds['transfer'].operation_ls(target_endpoint, path=target_path)
bnl_inputs = []
for file_meta in ls_result:
    if file_meta['type'] == "file":
        bnl_inputs.append("https://{}.e.globus.org/{}{}".format(target_endpoint, target_path, file_meta['name']))

## Generate a list of images to pass to the model for labeling
Here is an example image:

![example image](img/0000002f.jpg)

In [4]:
data = []
for filename in bnl_inputs[0:2]:
    print(filename)
    data.append({"file": filename, "token": creds['petrel'].access_token})
df_bnl = pd.DataFrame(data)
df_bnl.head()

https://e38ee745-6d04-11e5-ba46-22000b92c6ec.e.globus.org/Yager/model/raw/01470b9d_varied_sm/00000000.mat
https://e38ee745-6d04-11e5-ba46-22000b92c6ec.e.globus.org/Yager/model/raw/01470b9d_varied_sm/00000001.mat


,file,token
0,https://e38ee745-6d04-11e5-ba46-22000b92c6ec.e...,AgQEBzvJ2PrD6zGlBzQKaWO23YqNzD3XO1M476k9Bzv5Ky...
1,https://e38ee745-6d04-11e5-ba46-22000b92c6ec.e...,AgQEBzvJ2PrD6zGlBzQKaWO23YqNzD3XO1M476k9Bzv5Ky...


## Invoke the Yager model on one of these images

(This can be a bit slow if the result isn't cached. We are working to optimize.)

In [5]:
servable_name = "yager_xrd_classifier"
servable_id = dl.get_id_by_name(servable_name)

preds=[]

for d in tqdm(data, total=len(data)):
    res = dl.run(servable_id, {"data":[d]})
    preds.append(res[0][0])
df_bnl['prediction'] = preds
df_bnl.head()

HBox(children=(IntProgress(value=0, max=2), HTML(value='')))

,file,token,prediction
0,https://e38ee745-6d04-11e5-ba46-22000b92c6ec.e...,AgQEBzvJ2PrD6zGlBzQKaWO23YqNzD3XO1M476k9Bzv5Ky...,"[1.4985761254138197e-07, 0.999998927116394, 0...."
1,https://e38ee745-6d04-11e5-ba46-22000b92c6ec.e...,AgQEBzvJ2PrD6zGlBzQKaWO23YqNzD3XO1M476k9Bzv5Ky...,"[8.551788823751849e-07, 1.0, 8.665214323855253..."


## Translate predictions into tags
To make these predictions useful to humans...

In [6]:
df_tags = pd.read_csv('./tags.txt',  '\n', header=None) ## Eventually this will be read in from the model metadata in Search

def get_tags(entry):
    df_tmp = pd.DataFrame()
    df_tmp['probability'] = entry['prediction']
    df_tmp['tags'] = df_tags[0]
    return df_tmp

df_p = get_tags(df_bnl.iloc[0])
df_p.sort_values(["probability"], ascending=False).head(10)

,probability,tags
1,0.999999,Beam Off Image
12,0.000764,Ring
15,0.000229,Weak scattering
2,0.000222,Circular Beamstop
8,0.000077,Higher orders
16,0.000004,Wedge beamstop
3,0.000003,Diffuse high-q
6,0.000002,Halo
13,0.000001,Strong scattering
9,0.000001,Linear beamstop


The machine learning model predicts a 99.9% certainity that the beam is off image. We plan to incorporate this kind of model into the beamline facilities at ANL so that scientists (or robots!) at the beamline can quickly identify when their experiments are producing useless data.